## Wstępna analiza zbioru

Działania rozpoczęto od zaimportowania bibliotek `pandas` oraz `NumPy`.

In [ ]:
import pandas as pd

In [ ]:
import numpy as np

Następnie załadowano zbiór danych z pliku `TitanicMess.tsv`. Za pomocą argumentu `sep` wskazano separator, który został wykorzystany do oddzielania danych w zbiorze (tabulator), a za pomocą argumentu `dtype` narzucono typ danych dla kolumny `Name`.

In [ ]:
df = pd.read_csv("TitanicMess.tsv", sep = '\t', dtype={'name': np.str})

Podgląd pierwszych dziesięciu rekordów:

In [ ]:
df.head(10)

Zbiór składa się z 892 rekordów i 13 kolumn:

In [ ]:
df.shape

Lista kolumn wraz z typami danych, które przechowują:

In [ ]:
df.dtypes

Statystyka podsumowująca dla wszystkich kolumn w zbiorze:

In [ ]:
df.describe(include='all')

Analiza zbioru pod kątem brakujących wartości na poszczególnych kolumnach:

In [ ]:
df.isnull().sum()

Brakujące wartości występują na trzech kolumnach:
* `Age` - 173 brakujące wartości,
* `Cabin` - 685 brakujących wartości,
* `Embarked` - 2 brakujące wartości.

## Analiza poszczególnych kolumn

### 1. PassengerId

Wartości na tej kolumnie są identyfikatorami pasażerów, które powinny być kolejnymi liczbami naturalnymi (bez zera), a dodatkowo powinny być unikalne (każda wartość powinna wystąpić tylko raz). Nie ma brakujących wartości na tej kolumnie.

Okazuje się, że wartości są liczbami naturalnymi większymi od zera:

In [ ]:
np.logical_not(df['PassengerId'].astype(str).str.isdigit()).any()

In [ ]:
df['PassengerId'][df['PassengerId'] < 1].any()

Występują jednak duplikaty (rekordy o identyfikatorach 11 i 225):

In [ ]:
df['PassengerId'].is_unique

In [ ]:
df[df['PassengerId'].duplicated(keep=False)]

Należy także wspomnieć o tym, że ostatni rekord, w przeciwieństwie do pozostałych, ma identyfikator o wartości większej niż wskazywałaby na to liczba rekordów. Według "licznika" powinien mieć identyfikator 892, a ma 1000.

In [ ]:
df['PassengerId'][-3:]

Czyszczenie tej kolumny polegało na usunięciu duplikatów - opis wykonania tej czynności został przedstawiony w dalszej części sprawozdania.

### 2. Survived

Wartości na tej kolumnie dają informację o tym czy pasażer przeżył katastrofę: `0` - nie przeżył, `1` - przeżył, więc wartości powinny należeć do zbioru `{0, 1}`. Na tej kolumnie nie występują brakujące wartości.

In [ ]:
np.logical_not(df['Survived'].isin([0, 1])).any()

Wyżej udowodniono, że wszystkie wartości na kolumnie `Survived` należą do zbioru `{0, 1}`, więc ta kolumna nie wymaga przetwarzania.

### 3. Pclass

Kolumna `Pclass` zawiera informacje o klasie biletu pasażera, gdzie wartości powinny należeć do zbioru `{1, 2, 3}`. Według wcześniejszych analiz nie ma brakujących wartości na tej kolumnie.

In [ ]:
np.logical_not(df['Pclass'].isin([1, 2, 3])).any()

Okazuje się, że wszystkie wartości należą do zbioru `{1, 2, 3}`, więc ta kolumna nie wymaga przetwarzania.

### 4. Name

Kolumna `Name` zawiera informacje o nazwiskach pasażerów. Tutaj także nie występują brakujące wartości, ale za to występują niekiedy znaki, które raczej nie powinny znajdować się w nazwiskach, takie jak: `$` czy `&`:

In [ ]:
df[df['Name'].str.contains('[^A-Za-z\s"\'.,()/-]')]

Znaki akceptowane na tej kolumnie: wielka litera, mała litera, biały znak (spacja), cudzysłów, apostrof, kropka, przecinek, lewy nawias okrągły, prawy nawias okrągły, slash oraz myślnik.

Czyszczenie tej kolumny polegało na usunięciu znaków `$` oraz `&` - opis wykonania ten czynności został przedstawiony w dalszej części sprawozdania.

### 5. Sex

Wartości na kolumnie `Sex` oznaczają płeć pasażera, czyli powinny należeć do zbioru `{male, female}`. Okazuje się jednak, że tak nie jest:

In [ ]:
np.logical_not(df['Sex'].isin(['male', 'female'])).any()

In [ ]:
df['Sex'].value_counts()

Inne wartości prawdopodobnie wynikają z błędów podczas wprowadzania danych. Naprawianie tych błędów polegało na zamianie błędnych wartości na poprawne wartości. Opis wykonania tej czynności został przedstawiony w dalszej części sprawozdania.

### 6. Age

Wartości na kolumnie `Age` oznaczają wiek pasażera. Ze wstępnej analizy wynika, że występują tam brakujące wartości. Jednak nie jest to jedyny problem, ponieważ występują tam także wartości zmiennoprzecinkowe, co raczej jest błędem w przypadku informacji o wieku pasażera.

Na tej kolumnie zdarzają się wartości typu `0,75`, `0,83`, `0,92`:

In [ ]:
df[df['Age'].str.contains(r'^0,\d+$', na=False)]

Występuje też wartość `.9`:

In [ ]:
df[df['Age'].str.contains(r'^\.9$', na=False)]

Występują także wartości typu: `7,5`, `15,5`:


In [ ]:
df[df['Age'].str.contains('^\d+,5$', na=False)]

Zdarzają się także liczby ujemne, np. `-12`:

In [ ]:
df[df['Age'].str.contains('^-\d+$', na=False)]

Czyszczenie tej kolumny będzie polegało na usuwaniu całej wartości albo jej wybranej części (w zależności od wartości) - opis wykonania tych czynności został przedstawiony w dalszej części sprawozdania.

### 7. SibSp

Wartości na kolumnie `SibSp` oznaczają liczbę rodzeństwa/małżonków pasażera na pokładzie statku. Według wstępnej analizy nie ma tutaj brakujących wartości, a intuicja mówi, że powinny się tutaj pojawiać wartości będące liczbami naturalnymi (wraz z zerem).

In [ ]:
np.logical_not(df['SibSp'].astype(str).str.isdigit()).any()

In [ ]:
df['SibSp'][df['SibSp'] < 0].any()

Okazuje się, że wszystkie wartości na kolumnie `SibSp` są liczbami naturalnymi (wraz z zerem), czyli kolumna nie wymaga przetwarzania.

### 8. Parch

Wartości na kolumnie `Parch` oznaczają liczbę rodziców/dzieci pasażera na pokładzie statku. Według wstępnej analizy nie ma tutaj brakujących wartości, a intuicja mówi, że powinny się tutaj pojawiać wartości będące liczbami naturalnymi (wraz z zerem).

In [ ]:
np.logical_not(df['Parch'].astype(str).str.isdigit()).any()

In [ ]:
df['Parch'][df['Parch'] < 0].any()

Okazuje się, że wszystkie wartości na kolumnie `Parch` są liczbami naturalnymi (wraz z zerem), czyli kolumna nie wymaga przetwarzania.

### 9. Ticket

Wartości na kolumnie `Ticket` są numerami biletów pasażerów. Nie występują tutaj braki w danych. Wartości występujące na tym atrybucie można podzielić na dwie grupy: albo są to same liczby albo połączenie oznaczenia tekstowego wraz z liczbą. Ciężko określić czy numery biletów bez części tekstowej są poprawne albo niepoprawne. Wydaje się, że nie ma potrzeby przetwarzania/czyszczenia danych na tej kolumnie.

### 10. Fare

Wartości na kolumnie `Fare` reprezentują ceny biletów. Nie ma tutaj brakujących wartości. Dane powinny przyjmować postać liczb zmiennoprzecinkowych. Jednak występują wyjątki:

In [ ]:
df[df['Fare'].str.contains('[^0-9,]+')]

Czyszczenie danych na tej kolumnie polegało na usunięciu znaków innych niż cyfry i przecinki - opis wykonania tej czynności został przedstawiony w dalszej części sprawozdania.

### 11. Cabin

Ze względu na to, że na kolumnie `Cabin` brakuje ponad 75% wartości, a ich sztuczne uzupełnianie raczej mija się z celem, postanowiono usunąć tę kolumnę - opis wykonania tej czynności został przedstawiony w dalszej części sprawozdania.

### 12. Embarked

Wartości na atrybucie `Embarked` oznaczają skrótową nazwę portu, z którego pasażer rozpoczął podróż: `C - Cherbourg`, `Q - Queenstown`, `S - Southampton`. Z wstępnej analizy wynika, że na tym atrybucie brakuje tylko dwóch wartości. Każdy rekord powinien przyjąć jedną z trzech wartości: `C`, `Q`, `S`, jednak tak nie jest:

In [ ]:
np.logical_not(df['Embarked'].isin(['C', 'Q', 'S'])).any()

In [ ]:
df['Embarked'].value_counts()

Inne wartości prawdopodobnie wynikają z błędów podczas wprowadzania danych. Naprawianie tych błędów polegało na zamianie błędnych wartości na poprawne wartości. Opis wykonania tej czynności został przedstawiony w dalszej części sprawozdania.

### 13. Ship

Na kolumnie `Ship` występuje tylko jedna wartość dla wszystkich rekordów:

In [ ]:
df['ship'].value_counts()

Z tego powodu postanowiono usunąć tą kolumnę - opis wykonania tej czynności został przedstawiony w dalszej części sprawozdania.

## Czyszczenie/przetwarzanie danych

### 1.  Usuwanie duplikatów

Z wcześniejszej analizy wynika, że duplikaty występują jedynie na kolumnie `PassengerId`. Ich usunięcia dokonano za pomocą metody `drop_duplicates`, wskazując kolumnę (`subset`) oraz tryb (`keep`), gdzie określono, że pierwsze wystąpienie zduplikowanego rekordu ma nie być usuwane:

In [ ]:
df.drop_duplicates(subset='PassengerId', keep='first', inplace=True)

W wyniku czego pozbyto się duplikatów na kolumnie `PassengerId`:

In [ ]:
df['PassengerId'].is_unique

### 2. Poprawianie błędnych wartości

Poprawianie błędnych danych zazwyczaj sprowadza się podmienienia konkretnej
wartości na inną wartość albo do wykorzystania wyrażeń regularnych w celu odnalezienia pasujących do wyrażenia wartości i ich zastąpieniu.

### 2.1. Usuwanie niepoprawnych znaków z nazwisk

Z wcześniejszej analizy wynika, że w wartościach na kolumnie `Name` występują znaki `$` oraz `&`. Zdecydowano się je usunąć:

In [ ]:
df['Name'] = df['Name'].str.replace('[^A-Za-z\s"\'.,()/-]', '')

Brak wartości `$` oraz `&` na kolumnie `Name`:

In [ ]:
df['Name'].str.contains('[^A-Za-z\s"\'.,()/-]').any()

### 2.2. Błędy w zapisie płci

Wcześniejsza analiza wykazała, że w kilku wartościach na kolumnie `Sex` występują błędnie zapisane płcie. Błędy te zostały poprawione w następujący sposób:

In [ ]:
df['Sex'].replace(['mal', 'malef'], 'male', inplace=True)
df['Sex'].replace(['femmale', 'fem'], 'female', inplace=True)

W rezultacie udało się usunąć błędne wartości:

In [ ]:
df['Sex'].value_counts()

### 2.3. Błędy w zapisie wieku

Z wcześniejszej analizy wynika, że na kolumnie `Age` występują wartości zmiennoprzecinkowe, co nie jest pożądane w przypadku informacji o wieku.

Wartości typu `0,75`, `0,83` czy `0,92` zostają usunięte:

In [ ]:
df['Age'] = df['Age'].str.replace(r'^0,\d+$', 'NA')

Tak samo jak wartość `.9`:

In [ ]:
df['Age'] = df['Age'].str.replace(r'^\.9$', 'NA')

Z wartości typu `7,5`, `15,5` zostaje usunięta część ułamkowa:

In [ ]:
df['Age'] = df['Age'].str.replace(r'^(\d+),5$', r'\1')

Z liczb ujemnych zostaje usunięty znak `-`:

In [ ]:
df['Age'] = df['Age'].str.replace(r'^-(\d+)$', r'\1')

Na koniec wartości `NA` zostają przekonwertowane na `np.nan` w celu późniejszego uzupełnienia:

In [ ]:
df['Age'].replace('NA', np.nan, inplace=True)

Uznano także, że należy zmienić wartości powyżej `100` (w oryginalnym zbiorze występują wartości `250` oraz `4435`), ponieważ wynikają prawdopodobnie z omyłkowego dopisania cyfr. I tak wartość `250` staje się wartością `25`, a wartość `4435` staje się wartością `44`:

In [ ]:
df_age_without_nan = df[df['Age'].notnull()]
for index in df_age_without_nan[df_age_without_nan['Age'].astype(int) > 100].index:
    df.loc[index, 'Age'] = df.loc[index, 'Age'][0:2]

Wszystkie wartości na kolumnie `Age` (z wyłączeniem `NaN`) mają odpowiedni format:

In [ ]:
df['Age'].dropna().str.contains('[^\d{,2}]').any()

### 2.4. Oczyszczenie wartości na kolumnie `Fare`

Podczas analizy zauważono, że na kolumnie `Fare` występują niedozwolone znaki. Ich usunięcia dokonano w następujący sposób:

In [ ]:
df['Fare'] = df['Fare'].str.replace('[^0-9,]+', '')

W celu uniknięcia problemów podczas importu danych w dalszych etapach prac nad zbiorem zdecydowano się także na zastąpienie separatora liczb zmiennoprzecinkowych z `,` na `.` oraz sformatowanie wartości na typ `float`:

In [ ]:
df['Fare'] = df['Fare'].str.replace(',', '.')
df['Fare'] = df['Fare'].astype(float)

### 2.5. Błędy w nazwach portu startowego

Wcześniejsza analiza wykazała, że występują niepoprawne nazwy portów, z których pasażerowie rozpoczynali podróż. W celu naprawienia tych błędów zamieniono błędne nazwy na poprawne:

In [ ]:
df.replace({'Embarked': {'Co': 'C', 'Qe': 'Q', 'So': 'S'}}, inplace=True)

Wartości na kolumnie `Embarked` po poprawieniu błędów:

In [ ]:
df['Embarked'].value_counts()

### 3. Uzupełnianie brakujących wartości

W zbiorze brakujące wartości występują na trzech kolumnach: `Age`, `Cabin` oraz `Embarked`. Ze względu na brak ponad 75% wartości na kolumnie `Cabin` postanowiono ją usunąć, ale wartości na pozostałych kolumnach zostały uzupełnione.

### 3.1. Uzupełnianie wartości na kolumie `Age`

Postanowiono uzupełnić brakujące wartości na kolumnie `Age` średnim wiekiem występującym na tej kolumnie:

In [ ]:
mean_age = round(df['Age'].dropna().astype(int).mean())
print('Średni wiek: {0}'.format(mean_age))

In [ ]:
df['Age'].fillna(mean_age, inplace=True)

Wszystkie brakujące wartości zostały uzupełnione:

In [ ]:
df['Age'].isnull().any()

### 3.2. Uzupełnianie wartości na kolumnie `Embarked`

W tym przypadku dwie brakujące wartości postanowiono uzupełnić wartością najczęściej występującą na kolumnie `Embarked`:

In [ ]:
df['Embarked'].value_counts()

In [ ]:
most_popular_embarked_value = df['Embarked'].value_counts().index[0]
print('Wartość najczęściej występująca: {0}'.format(most_popular_embarked_value))

In [ ]:
df['Embarked'].fillna(most_popular_embarked_value, inplace=True)

Wszystkie brakujące wartości zostały uzupełnione:

In [ ]:
df['Embarked'].isnull().any()

### 4.  Usuwanie kolumn

Zdecydowano się na usunięcie kolumn `Cabin` - ponad 75% brakujących wartości oraz `Ship` - jedna wartość występująca na wszystkich rekordach.

In [ ]:
df.drop(columns=['Cabin', 'ship'], axis=1, inplace=True)

Potwierdzenie usunięcia kolumn:

In [ ]:
'Cabin' in df

In [ ]:
'ship' in df

## Zapis przetworzonych danych do pliku

Po przetworzeniu wszystkich kolumn nie ma już brakujących wartości:

In [ ]:
df.isnull().sum()

Na koniec zapisano oczyszczone dane do pliku `TitanicCleaned.tsv`:

In [ ]:
df.to_csv('TitanicCleaned.tsv', sep='\t', index=False)